In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 7 - Introduction to clustering

In this notebook we will see some techniques for clustering.
Here, we will see only k-means, in the next session we will keep working on clustering and we will see some other methods as well.

The notebook is divided in two parts:
- firstly, we will be generate some artificial data to look at how k-means works
- then, we will move to a real dataset (not used in previous sessions) and we will apply k-means to that


# Index

- [0. imports](#0.)
- [1. introduction to k-means](#1.)
    - [1.1 Using the make_blobs method from sklearn](#1.1)
    - [1.2 Changing the distribution](#1.2)
    - [1.3 Changing the variance](#1.3)
    - [1.4 Changing the size and density](#1.4)
- [2. KMeans on a real intrusion detection dataset](#2.)
    - [2.1 Load and explore the dataset](#2.1)
    - [2.2 Mapping attacks to categories](#2.2)
    - [2.3 Using scatter plots to analyze feature distribution](#2.3)
    - [2.4 Prepare data for clustering](#2.4)
    - [2.5 Perform the actual clustering](#2.5)
    - [2.6 Data exploration after clustering](#2.6)
    - [2.7 Using t-SNE](#2.7)
    - [2.8 Evaluation](#2.8)
    - [2.9 Using different values of K](#2.9)
    

# 0.

## Imports
[Index](#Index)

In [ ]:
import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

# 1.
## Introduction to KMeans

[Index](#Index)

This example is inspired by [this one](https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_assumptions.html#sphx-glr-auto-examples-cluster-plot-kmeans-assumptions-py) from the scikit-learn website.

## 1.1
### Using the `make_blobs` method from sklearn
[Index](#Index)

In order to see how KMeans works in practice, we will start from some artificial 2D data.

In [ ]:
# definition of some variables which we will need in the next few cells

# list of colors
colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:cyan', 'tab:purple', 'tab:gray', 'tab:olive']
s = 20   # size of scatter plots
a = 0.5  # transparency of scatter plots

The first step is the creation of three separate blobs of artificial data.
They are pseudo-random, have a regular shape and are fairly far from each other.

In [ ]:
n_samples = 1500  # number of random points to generate
random_state = 170  # random seed, for reproducibility

# to generate the random blobs
X, y = make_blobs(n_samples=n_samples, random_state=random_state)

# plot them
fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(X[:, 0], X[:, 1], s=s, alpha=a, c='darkred')
plt.show()

Let's now use KMeans to perform clustering.
By looking at the data, we can see that it was generated in order to have three blobs.
Let's start with 3 clusters, then.

KMeans is provided in scikit-learn and it can trained in the same way as you would train a classifier (using `fit` and `predict`).
However, since we perform the `predict` on the same data used for fitting it (as there is no test data), we can directly use the `fit_predict` method.


In [ ]:
n_clusters = 3
y_pred = KMeans(n_clusters=n_clusters, random_state=random_state).fit_predict(X)

Let's now print the same blobs as above, but using different colours depending on the cluster that each point belongs to (according to the KMeans model we have just trained).

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(X[:, 0], X[:, 1], s=s, alpha=a, c=[colors[idx] for idx in y_pred])
ax.set_title("n_clusters = %d" % n_clusters)
plt.show()

Everything works as expected.

However, if we use a different number of clusters, then things start to get a bit leass neat.

In [ ]:
n_clusters = 2
y_pred = KMeans(n_clusters=n_clusters, random_state=random_state).fit_predict(X)

fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(X[:, 0], X[:, 1], s=s, alpha=a, c=[colors[idx] for idx in y_pred])
ax.set_title("n_clusters = %d" % n_clusters)
plt.show()

In [ ]:
n_clusters = 4
y_pred = KMeans(n_clusters=n_clusters, random_state=random_state).fit_predict(X)

fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(X[:, 0], X[:, 1], s=s, alpha=a, c=[colors[idx] for idx in y_pred])
ax.set_title("n_clusters = %d" % n_clusters)
plt.show()

In [ ]:
n_clusters = 5
y_pred = KMeans(n_clusters=n_clusters, random_state=random_state).fit_predict(X)

fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(X[:, 0], X[:, 1], s=s, alpha=a, c=[colors[idx] for idx in y_pred])
ax.set_title("n_clusters = %d" % n_clusters)
plt.show()

## 1.2
### Changing the distribution
[Index](#Index)

We can also try with different distributions of data, and blobs with different shapes, densities, and sizes.

In [ ]:
transformation = [[0.6, -0.6], [-0.4, 0.8]]
X_transformed = np.dot(X, transformation)

fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(X_transformed[:, 0], X_transformed[:, 1], s=s, alpha=a, c='darkred')
ax.set_title("Original data")
plt.show()

In [ ]:
list_n_clusters = [[2,3], [4,5]]

fig, ax = plt.subplots(2, 2, figsize=(12, 12))

for idx1 in [0,1]:
    for idx2 in [0,1]:
        y_pred = KMeans(n_clusters=list_n_clusters[idx1][idx2], random_state=random_state).fit_predict(X_transformed)
        ax[idx1][idx2].scatter(X_transformed[:, 0], X_transformed[:, 1], s=s, alpha=a, c=[colors[idx] for idx in y_pred])
        ax[idx1][idx2].set_title("n_clusters = %d" % list_n_clusters[idx1][idx2])

plt.show()

## 1.3
### Changing the variance
[Index](#Index)

In [ ]:
X_var, y_var = make_blobs(n_samples=n_samples, cluster_std=[1.0, 2.5, 0.5], random_state=random_state)

fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(X_var[:, 0], X_var[:, 1], s=s, alpha=a, c='darkred')
ax.set_title("Original data")
plt.show()

In [ ]:
list_n_clusters = [[2,3], [4,5]]

fig, ax = plt.subplots(2, 2, figsize=(12, 12))

for idx1 in [0,1]:
    for idx2 in [0,1]:
        y_pred = KMeans(n_clusters=list_n_clusters[idx1][idx2], random_state=random_state).fit_predict(X_var)
        ax[idx1][idx2].scatter(X_var[:, 0], X_var[:, 1], s=s, alpha=a, c=[colors[idx] for idx in y_pred])
        ax[idx1][idx2].set_title("n_clusters = %d" % list_n_clusters[idx1][idx2])

plt.show()

## 1.4
### Changing the size and density
[Index](#Index)

In [ ]:
X_filtered = np.vstack((X[y == 0][:500], X[y == 1][:100], X[y == 2][:10]))

fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(X_filtered[:, 0], X_filtered[:, 1], s=s, alpha=a, c='darkred')
ax.set_title("Original data")
plt.show()

In [ ]:
list_n_clusters = [[2,3], [4,5]]

fig, ax = plt.subplots(2, 2, figsize=(12, 12))

for idx1 in [0,1]:
    for idx2 in [0,1]:
        y_pred = KMeans(n_clusters=list_n_clusters[idx1][idx2], random_state=random_state).fit_predict(X_filtered)
        ax[idx1][idx2].scatter(X_filtered[:, 0], X_filtered[:, 1], s=s, alpha=a, c=[colors[idx] for idx in y_pred])
        ax[idx1][idx2].set_title("n_clusters = %d" % list_n_clusters[idx1][idx2])

plt.show()

You can see that KMeans is not always capable of creating the correct clusters, and that is due to its algorithm.
In the next session we will see some other clustering algorithms as well, to see how they compare on different distributions of data.

# 2.
## KMeans on a real intrusion detection dataset

[Index](#Index)

We are going to use a dataset which is an updated version of the KDD dataset we used in previous sessions.
The original KDD dataset is useful for practicing and getting familiar with techniques, but it has some intrisic problems, related to how the data was collected and labeled.
Go to [this webpage](https://www.unb.ca/cic/datasets/nsl.html) and have a look at the description of the new dataset to see how it addresses the problems of the KDD dataset (read [this paper](https://www.researchgate.net/publication/48446353_A_detailed_analysis_of_the_KDD_CUP_99_data_set), if you are intereseted in more details).

---

If you click on **[this link](http://205.174.165.80/CICDataset/NSL-KDD/Dataset/NSL-KDD.zip)** the download of the dataset should start.

If the link above doesn't work, follw these instructions:
- go to https://www.unb.ca/cic/datasets/nsl.html
- scroll to the end of the page, there is a link to the actual download;
- you will be redirected to another page asking for some information (there should be no check on the data you provide, so you can fill everything with *asd* if you want);
- download the NSL-KDD.zip file

---

Regardless of the link you used for downloading the dataset, you should now have an archive named *NSL-KDD.zip*; extract it in the folder of the notebook

You should now have a directory named NSL-KDD, containing several files. You have to focus on the following ones:
- *index.html*: contains a brief description of the files, you should read it
- *KDDTrain+.txt*: the file containing the training data
- *KDDTest+.txt*: the file containing the test data

In case you need it, there is also a reduced training set, stored in:
- *KDDTrain+_20Percent.txt*: reduced training set

---

## 2.1
### Load and explore the dataset

[Index](#Index)

#### Define the filename for the training data

In [ ]:
TRAIN_DATA_FILENAME = 'NSL-KDD/KDDTrain+.txt'

In [ ]:
# TRAIN_DATA_FILENAME = 'NSL-KDD/KDDTrain+_20Percent.txt'

#### Define the columns of the dataframe

In [ ]:
headers = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 
    'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 
    'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 
    'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 
    'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 
    'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 
    'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 
    'dst_host_srv_rerror_rate', 'class', 'difficulty_level'
]

#### Read the file

In [ ]:
train_df = pd.read_csv(TRAIN_DATA_FILENAME, names=headers)

<div class="alert alert-block alert-danger">
<b>Q: Display 5 random rows of the dataframe.</b>
</div>

<div class="alert alert-block alert-danger">
<b>Q: Display the first 5 rows of the dataframe.</b>
</div>

<div class="alert alert-block alert-danger">
<b>Q: Display the last 5 rows of the dataframe.</b>
</div>

<div class="alert alert-block alert-danger">
<b>Q: Show the columns of the dataframe</b>
</div>

<div class="alert alert-block alert-danger">
<b>Q: Print the number of rows of the dataframe</b>
</div>

<div class="alert alert-block alert-danger">
<b>Q: Print the number of columns of the dataframe</b>
</div>

<div class="alert alert-block alert-danger">
<b>Q: How many features are there in the original dataset?</b>
</div>

<div class="alert alert-block alert-danger">
<b>Q: Find the type of each feature (i.e. categorical, binary, numerical, etc.)</b>
</div>

<div class="alert alert-block alert-danger">
<b>Q: How many rows are there for each unique value of "root_shell"?</b>

\[It is a binary feature. if you though it was not a binary feature go back to the previous question and focus a bit more on that!\]
</div>

<div class="alert alert-block alert-danger">
<b>Q: How many rows are there for each unique value of "logged_in"?</b>
</div>

<div class="alert alert-block alert-danger">
<b>Q: How many rows are there for each unique value of "class"? Print and plot the distribution.</b>
</div>

## 2.2
### Mapping attacks to categories

[Index](#Index)

#### As in the previous sessions, we will now map each value of the 'class' column to 1 of 5 possible categories

In [ ]:
category_mapping = {
    'normal': 'benign',
    'back': 'dos',
    'buffer_overflow': 'u2r',
    'ftp_write': 'r2l',
    'guess_passwd': 'r2l',
    'imap': 'r2l',
    'ipsweep': 'probe',
    'land': 'dos',
    'loadmodule': 'u2r',
    'multihop': 'r2l',
    'neptune': 'dos',
    'nmap': 'probe',
    'perl': 'u2r',
    'phf': 'r2l',
    'pod': 'dos',
    'portsweep': 'probe',
    'rootkit': 'u2r',
    'satan': 'probe',
    'smurf': 'dos',
    'spy': 'r2l',
    'teardrop': 'dos',
    'warezclient': 'r2l',
    'warezmaster': 'r2l',
}

In [ ]:
train_df['attack_type'] = train_df.apply(lambda r: category_mapping[r['class']], axis=1)

<div class="alert alert-block alert-danger">
<b>Q: How many rows are there for each unique value of "attack_type"?</b>
</div>

<div class="alert alert-block alert-danger">
<b>Q: Plot the distribution of feature `duration`.</b>
</div>

The first bin contains almost all the samples. 

We can address this issue in several ways:
- using log scale
- plotting separately values above and below a `duration` threshold

<div class="alert alert-block alert-danger">
<b>Q: Print the possible unique values of "protocol_type", and plot the ditribution.</b>
</div>

<div class="alert alert-block alert-danger">
<b>Q: Print the distribution of the "src_bytes" attribute.</b>
</div>

<div class="alert alert-block alert-danger">
<b>Q: Print the distribution of the "src_bytes" attribute, separately for each "protocol_type".</b>
</div>

<div class="alert alert-block alert-danger">
<b>Q: Print the distribution of the "dst_bytes" attribute, separately for each "protocol_type".</b>
</div>

---

## 2.3
### Using scatter plots to analyze feature distribution

[Index](#Index)

By plotting with a scatter plot two attributes, we can look for correlations between them (and for rules taht might discriminate between different classes).

In [ ]:
x, y = train_df['src_bytes'].values, train_df['dst_bytes'].values

fig, ax = plt.subplots()
ax.scatter(x, y)
plt.show()

most of the points are very close to 0.0 (be careful with the scale of the axis: it is 1e9, which means 10^9 !), let's focus on them

In [ ]:
thres = 50000
tmp_df = train_df[(train_df['src_bytes']<=thres)&(train_df['dst_bytes']<=thres)]
x = tmp_df['src_bytes'].values
y = tmp_df['dst_bytes'].values

fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(x, y, s=5, alpha=0.5)

plt.show()

In [ ]:
thres = 1000
tmp_df = train_df[(train_df['src_bytes']<=thres)&(train_df['dst_bytes']<=thres)]
x = tmp_df['src_bytes'].values
y = tmp_df['dst_bytes'].values

fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(x, y, s=5, alpha=0.5)

plt.show()

Let's now do the same scatter plot but assigning a different color to each point, depending on the protocol_type

In [ ]:
c = []
for protocol in tmp_df['protocol_type'].values:
    if protocol == 'tcp':
        c.append('tab:green')
    elif protocol == 'udp':
        c.append('tab:orange')
    elif protocol == 'icmp':
        c.append('tab:blue')

In [ ]:
thres = 1000
tmp_df = train_df[(train_df['src_bytes']<=thres)&(train_df['dst_bytes']<=thres)]

fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(tmp_df['src_bytes'].values, tmp_df['dst_bytes'].values, s=5, alpha=0.5, c=c)

plt.show()

Alternatively, you could use the following plot for the scatter plot above, which is more convenient.

You can also repeat the same plot removing one at a time the protocol types to better observe the distribution (or you could plot them in three different subplots)

In [ ]:
thres = 1000
tmp_df = train_df[(train_df['src_bytes']<=thres)&(train_df['dst_bytes']<=thres)]

fig, ax = plt.subplots(figsize=(8, 8))

tmp_df_icmp = tmp_df[tmp_df['protocol_type']=='icmp']
ax.scatter(tmp_df_icmp['src_bytes'].values, tmp_df_icmp['dst_bytes'].values, s=5, alpha=0.5, c=colors[0], label='icmp')

tmp_df_udp = tmp_df[tmp_df['protocol_type']=='udp']
ax.scatter(tmp_df_udp['src_bytes'].values, tmp_df_udp['dst_bytes'].values, s=5, alpha=0.5, c=colors[1], label='udp')

tmp_df_tcp = tmp_df[tmp_df['protocol_type']=='tcp']
ax.scatter(tmp_df_tcp['src_bytes'].values, tmp_df_tcp['dst_bytes'].values, s=5, alpha=0.5, c=colors[2], label='tcp')

ax.legend()
ax.set_xlabel('src_bytes')
ax.set_ylabel('dst_bytes')
plt.show()

---

<div class="alert alert-block alert-danger">
<b>Q: Perform a similar analysis for the "duration" and the "src_bytes" attributes.</b> 
</div>

<div class="alert alert-block alert-danger">
<b>Q: Plot the "same_srv_rate" against the "diff_srv_rate", and analyse it.</b>
</div>

In this case you know the true labels (benign or attack), so you could even perform the scatter plot separately for malicious samples and benign samples.

In [ ]:
df_benign = train_df[train_df['attack_type']=='benign']
df_attack = train_df[train_df['attack_type']!='benign']

fig, ax = plt.subplots(1, 2, figsize=(14, 6))

ax[0].scatter(df_benign['same_srv_rate'].values, df_benign['diff_srv_rate'].values, s=2, alpha=0.7, c='g', label='benign')
ax[0].set_xlabel('same_srv_rate')
ax[0].set_ylabel('diff_srv_rate')
ax[0].legend()

ax[1].scatter(df_attack['same_srv_rate'].values, df_attack['diff_srv_rate'].values, s=2, alpha=0.7, c='r', label='attack')
ax[1].set_xlabel('same_srv_rate')
ax[1].set_ylabel('diff_srv_rate')
ax[1].legend()

plt.show()

It looks like the malicious samples are much more frequent than the benign samples, but actually that is not the case:

In [ ]:
print("Num benign samples = %d" % len(df_benign))
print("Num malicious samples = %d" % len(df_attack))

This suggests that the benign samples are much more "standard" than the attacks, in the sense that they tend to frequently have similar values (at least for these two attributes we have observed here).

---

## 2.4
### Prepare data for clustering

[Index](#Index)

In order to be able to perform clustering, we have to perform the usual preprocessing of the data:
- one hot encoding
- scaling

If we don't do this we might encounter some problems (e.g. some features are ignored because the possible range of values is much lower than others)

In [ ]:
col_names = np.array(headers)

nominal_idx = [1, 2, 3]
binary_idx = [6, 11, 13, 14, 20, 21]
numeric_idx = list(set(range(41)).difference(nominal_idx).difference(binary_idx))

nominal_cols = col_names[nominal_idx].tolist()
binary_cols = col_names[binary_idx].tolist()
numeric_cols = col_names[numeric_idx].tolist()

In [ ]:
print("Nominal cols:\n", nominal_cols, "\n")
print("Binary cols:\n", binary_cols, "\n")
print("numeric_cols:\n", numeric_cols, "\n")

<div class="alert alert-block alert-info">
<b>Some of the clustering algorithms tend to be intractable on small machines as the dimensionality increases.
Thus, in order to (hopefully) avoid problems on your machine, I remove here the 'service' column, which is a categorical one and increases a lot the dimensionality of the dataset</b>
</div>

In [ ]:
train_df = train_df.drop('service', axis=1)
nominal_cols = [x for x in nominal_cols if x != 'service']

<div class="alert alert-block alert-danger">
<b>Q: Perform one hot encoding of the nominal cols</b>
</div>

In [ ]:
train_df = # TODO

<div class="alert alert-block alert-danger">
<b>Q: Perform scaling with the StandardScaler</b>
</div>

In [ ]:
standard_scaler = # TODO
train_df[numeric_cols] = # TODO

---

## 2.5
### Perform the actual clustering

[Index](#Index)

While performing clustering, we do not want information about the label, since it is information that is missing in unseen data (it is the target label).
So we will fit the clustering model on the dataframe after dropping such columns, as in:
```
    train_df.drop(['class', 'attack_type', 'difficulty_level'], axis=1)
```

- training the clustering algorithm

In [ ]:
t0 = time.time()
kmeans = KMeans(n_clusters=2, random_state=random_state).fit(
    train_df.drop(['class', 'attack_type', 'difficulty_level'], axis=1)
)
print("Elapsed time = %.4f s" % (time.time()-t0))

- you can print the centers of the clusters (which are, in this case, two 52-D points) 

In [ ]:
kmeans.cluster_centers_

- you can print the labels for the elements in DF; these labels represent the ID of the clusters found with KMeans

In [ ]:
kmeans.labels_

- let's save those labels in a new column of the dataframe

In [ ]:
train_df['cluster'] = kmeans.labels_

## 2.6
### Data exploration after clustering

[Index](#Index)

We can explore the data to visualize the clusters (plotting two features at a time) and understand how the KMeans algorithm worked.
We can also plot the original classes (since in this case they were known to obsserve whether KMeans managed to separate them in different clusters).

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))

ax.scatter(train_df['same_srv_rate'].values, train_df['diff_srv_rate'].values, s=5, alpha=0.7, c=[colors[idx] for idx in train_df['cluster']])
ax.set_xlabel('same_srv_rate')
ax.set_ylabel('diff_srv_rate')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))

ax.scatter(train_df['src_bytes'].values, train_df['dst_bytes'].values, s=5, alpha=0.7, c=[colors[idx] for idx in train_df['cluster']])
ax.set_xlabel('src_bytes')
ax.set_ylabel('dst_bytes')
# ax.set_xlim(-0.02, 0.0)
# ax.set_ylim(-0.02, 0.01)
plt.show()

## 2.7
### Using t-SNE

[Index](#Index)


The problem here is that we cannot plot more than two (possibly three) features. 

However, we might want to use [t-SNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html#sklearn.manifold.TSNE) to better visualize the clusters.

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
X_embedded = TSNE(n_components=2, perplexity=20).fit_transform(train_df.drop(['class', 'attack_type', 'difficulty_level'], axis=1))

In [ ]:
X_embedded.shape

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(18, 9))

ax[0].set_title("KMeans clusters (k=2)")
ax[0].scatter(X_embedded[:,0], X_embedded[:,1], s=5, alpha=0.7, c=[colors[idx] for idx in train_df['cluster']])

ax[1].set_title("Benign vs. malicious samples")
ax[1].scatter(X_embedded[:,0], X_embedded[:,1], s=5, alpha=0.7, c=['g' if attack=='benign' else 'r' for attack in train_df['attack_type']])

plt.show()

## 2.8
### Evaluation

[Index](#Index)

We will see the metrics for evaluating clustering in the next session, but here we can get an intuition of how well the clustering worked by looking at the true label (which in our case was known).

<div class="alert alert-block alert-danger">
<b>Q: In this case, we know the ground truth (i.e. the attack_type). Try to evaluate the accuracy of clustering by looking at the attack types of the entries of each cluster.</b>
</div>

e.g. after a perfect clustering, I'd have in each cluster only elements belonging to one attack_type

## 2.9
### Using different values of K

[Index](#Index)

<div class="alert alert-block alert-danger">
<b>Q: Repeat the previous analysis using a different number of clusters.</b>
</div>

---